In [ ]:
import pathlib
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pooch
import requests
import pyproj
import geojson
import rasterio
import pysheds, pysheds.grid, pysheds.view
import firedrake
import icepack

First, we'll download a 1/3-arcsecond DEM of the region we're interested in.
The DEM comes from the Oregon Lidar Consortium, which is part of the Oregon Department of Geology and Mineral Industries (DOGAMI).
The DOGAMI website has an interactive online [viewer](https://gis.dogami.oregon.gov/maps/lidarviewer/) and download utility for LiDAR data.
The code below uses a library called [pooch](https://www.fatiando.org/pooch) to describe what file we want to get and from where.

In [ ]:
url = "https://www.oregongeology.org/pubs/ldq/"
archive_filename = "LDQ-43124D1.zip"
checksum = "cb1fcb26fbb6e84640a554fb2287c619cfe6f54bc81a6423624273ceb21f7647"
dem = pooch.create(
    path=pooch.os_cache("hillslope"),
    base_url=url,
    registry={archive_filename: checksum},
)

Next we'll actually fetch the raw data, unzip it, and extract a `.adf` file (an ArcInfo binary format) containing the actual DEM.

In [ ]:
try:
    downloader = pooch.HTTPDownloader(progressbar=True)
    files = dem.fetch(
        archive_filename,
        processor=pooch.Unzip(),
        downloader=downloader,
    )
except requests.exceptions.SSLError:
    downloader = pooch.HTTPDownloader(progressbar=True, verify=False)
    files = dem.fetch(
        archive_filename,
        processor=pooch.Unzip(),
        downloader=downloader,
    )

In [ ]:
filename = [
    f for f in files if "South Coast" in f and "Bare_Earth" in f and "w001001.adf" in f
][0]

print(filename)

The region we're interested in is near 43.464N, 124.119W (see the caption to figure 5 of Roering 2008).
We'll look at everything within a few hundred feet to make sure we get the whole study area.

In [ ]:
oregon_gic_lambert = pyproj.CRS(3644)
lat_lon = pyproj.CRS(4326)
transformer = pyproj.Transformer.from_crs(lat_lon, oregon_gic_lambert)

lon, lat = -124.119, 43.464
x, y = transformer.transform(lat, lon)

Lx, Ly = 2000.0, 2000.0

We'll extract only a small window around the study area so that we don't waste a ton of computing time later calculating upslope areas that we won't use.

In [ ]:
source = rasterio.open(filename, "r")
transform = source.transform
window = (
    rasterio.windows.from_bounds(x - Lx, y - Ly, x + Lx, y + Ly, transform)
    .round_lengths()
    .round_offsets()
)
dem = source.read(indexes=1, window=window, masked=True)
left, bottom, right, top = rasterio.windows.bounds(window, transform)
transform = rasterio.windows.transform(window, transform)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
extent = (left, right, bottom, top)
colors = ax.imshow(dem, vmin=0.0, vmax=1200.0, extent=extent)
ax.scatter([x], [y], color="tab:red")
fig.colorbar(colors);

Next we'll use the pysheds package to compute the catchment areas.
The steps here are to (1) add the elevation data, (2) fill depressions that won't drain out of the domain, (3) remove flat parts of the DEM where a flow direction can't meaningfully be defined, (4) calculate flow directions using the D${}^\infty$ routing algorithm from [Tarboton 1997](https://doi.org/10.1029/96WR03137), and (5) calculate the accumulation or catchment area.

In [ ]:
viewfinder = pysheds.view.ViewFinder(affine=transform, shape=dem.shape, crs=oregon_gic_lambert)
raster = pysheds.view.Raster(dem, viewfinder)
grid = pysheds.grid.Grid(viewfinder=viewfinder).from_raster(raster)
flooded_elevation = grid.fill_depressions(raster)
inflated_elevation = grid.resolve_flats(flooded_elevation)
flow_dir = grid.flowdir(inflated_elevation, routing="dinf")
accumulation = grid.accumulation(flow_dir, routing="dinf")

The accumulation area is best viewed on a logarithmic scale.
The bright yellow areas are ridge tops, and the dark blue areas are valleys and often rivers.
Moreover, the Roering 2008 paper specifies that we want to focus only on the parts of the domain where the accumulation area is less than 250 m${}^2$ = 3.284${}^2$ $\times$ 250 ft${}^2$.

In [ ]:
meters_to_feet = 3.284
vmax = meters_to_feet**2 * 250

fig, axes = plt.subplots()
norm = LogNorm(vmin=1, vmax=vmax + 1)
image = axes.imshow(accumulation + 1, extent=extent, cmap="viridis_r", norm=norm)
fig.colorbar(image);

The next plot shows the elevation with the valleys masked out.

In [ ]:
mask = accumulation > vmax
elevation_masked = ma.masked_array(raster, mask=mask)

fig, axes = plt.subplots()
axes.ticklabel_format(axis="both", style="scientific", scilimits=(0, 0))
image = axes.imshow(elevation_masked, extent=extent)
fig.colorbar(image);

The next step is to digitize the outline of the domain that we want to simulate into some vector format.
First, we'll save the DEM and catchment area to GeoTIFF files so that we can open them in a GIS.

In [ ]:
profile = {
    "driver": "GTiff",
    "count": 1,
    "height": dem.shape[0],
    "width": dem.shape[1],
    "crs": "EPSG:3644",
    "transform": transform,
    "dtype": np.float64,
}

with rasterio.open("elevation.tif", "w", **profile) as destination:
    destination.write(dem, indexes=1)

with rasterio.open("catchment.tif", "w", **profile) as destination:
    destination.write(accumulation, indexes=1)

Outside of this notebook, I opened the DEM and catchment area files in a GIS, manually traced out the boundaries of the hillslope we're interested in, and saved them to GeoJSON files.

In [ ]:
input_files = pathlib.Path("./").glob("sullivan-creek[0-9].geojson")
outlines = []
for filename in input_files:
    with open(filename, "r") as outline_file:
        outline = geojson.load(outline_file)
        outlines.append(outline)

The final step is to mesh the outline.
The code below creates one feature collection from all of the outlines and does a bit of cleanup on them so that it's easy to detect where one segment of the boundary matches up with another segment.

In [ ]:
features = sum((outline["features"] for outline in outlines), [])
crs = {"type": "name", "properties": {"name": "urn:ogc:def:crs:EPSG::3644"}}
collection = geojson.FeatureCollection(features, crs=crs, name="sullivan-creek")
outline = icepack.meshing.normalize(collection)

The meshing module in icepack includes a routine to turn a normalized GeoJSON outline into the input format for the mesh generator gmsh.
The file extension for gmsh geometry files is `.geo`.

In [ ]:
geometry = icepack.meshing.collection_to_geo(outline)
with open("sullivan-creek.geo", "w") as geometry_file:
    geometry_file.write(geometry.get_code())

Then we can make a command-line call to gmsh to triangulate the interior of the outline.

In [ ]:
!gmsh -2 -v 0 -format msh2 -o sullivan-creek.msh sullivan-creek.geo

Now we can read that outline into a Firedrake mesh object.

In [ ]:
mesh = firedrake.Mesh("sullivan-creek.msh")

In [ ]:
fig, axes = plt.subplots()
norm = LogNorm(vmin=1, vmax=vmax + 1)
image = axes.imshow(accumulation + 1, extent=extent, cmap="viridis_r", norm=norm)
fig.colorbar(image)

coords = mesh.coordinates.dat.data_ro
δ = 250.0
axes.set_xlim((coords[:, 0].min() - δ, coords[:, 0].max() + δ))
axes.set_ylim((coords[:, 1].min() - δ, coords[:, 1].max() + δ))
firedrake.triplot(mesh, interior_kw={"linewidth": 0.1}, axes=axes);

Now that we have a workable geometry for the spatial domain, we can start on the modeling.